In [2]:
from env import *
from env_tradestation import *
from data_retriever_simple import Client
import acquire
import prepare

import cbpro
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import RFE, SelectKBest

import warnings
warnings.filterwarnings("ignore")

np.random.seed(123)

# Compare the ability to predict cryptocurrency returns using machine learning and ARIMA models

###  Acquire Data Using Coinbase Pro API (account and API key required)

In [3]:
results = acquire.get_data_from_csv()

Max first date is 2016-08-17 00:00:00, starting all dataframes at this day
Corrected btc low data for 2017-04-15


### Prepare Data:
- Data comes pretty clean from Coinbase
- The min low value for the dataset was suspiciously low. Further investigation found that for 2017-04-15 errors with Coinbase caused this low value. 

In [6]:
prepared_results = prepare.prepare_crypto_data(results)

Max first date is 2016-08-17 00:00:00, starting all dataframes at this day
Corrected btc low data for 2017-04-15


### Bitcoin prices explored and modeled

In [4]:
btc=prepared_results['BTC_USD']

In [5]:
btc

,time,low,high,open,close,volume
time,,,,,,
2016-08-17,2016-08-17,569.67,581.99,579.02,574.45,4509.774374
2016-08-18,2016-08-18,573.00,576.99,574.45,574.09,3964.998989
2016-08-19,2016-08-19,572.68,576.99,574.05,573.67,4038.303001
2016-08-20,2016-08-20,572.38,583.92,573.68,581.87,3581.224253
2016-08-21,2016-08-21,579.34,583.99,581.72,581.29,2442.633748
...,...,...,...,...,...,...
2022-04-18,2022-04-18,38550.00,41100.00,39681.11,40803.58,13090.794234
2022-04-19,2022-04-19,40571.99,41772.94,40810.87,41503.85,10218.113707
2022-04-20,2022-04-20,40900.00,42209.58,41501.56,41377.61,14609.283927
